In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# <span style="color:red"> Step.1_PDF (과실비율인정기준) 데이터 추출 </span>

In [2]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출
    - 사고유형코드, 기본과실비율, 사고유형명, 관련판례 등 추출
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
                
            # 페이지별 데이터 추출 시도
            page_data = extract_page_data_standards(text, page_num + 1)
            if page_data:
                extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def extract_page_data_standards(text, page_num):
    """
    과실비율인정기준 PDF의 각 페이지에서 데이터 추출
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:500]  # 디버깅용 원본 텍스트 일부
    }
    
    lines = text.split('\n')
    
    # 1. 사고유형코드 찾기 (예: "보1", "차1" 등)
    for line in lines:
        # "보1", "보2", "차1", "차2" 패턴 찾기
        code_match = re.search(r'([보차]\d+)', line)
        if code_match:
            result['accident_code'] = code_match.group(1)
            # 사고유형명도 같은 줄에서 찾기
            # 예: "보1  보행자 직선신호 횡단게시, 직선신호 충격 사고"
            parts = line.split()
            if len(parts) > 1:
                result['accident_name'] = ' '.join(parts[1:])
            break
    
    # 2. 기본과실비율 찾기 (숫자 패턴)
    for line in lines:
        # "70", "기본과실비율 70" 같은 패턴
        ratio_match = re.search(r'기본과실비율.*?(\d+)', line)
        if ratio_match:
            result['basic_fault_ratio'] = ratio_match.group(1)
            break
        # 단순히 숫자만 있는 경우도 체크
        elif re.search(r'^\s*\d{1,2}\s*$', line):
            result['basic_fault_ratio'] = line.strip()
    
    # 3. 수정요소 찾기 (①, ②, ③ 패턴)
    modification_list = []
    for line in lines:
        mod_match = re.search(r'[①②③④⑤]\s*(.+?)\s*([+\-]\d+)', line)
        if mod_match:
            factor = mod_match.group(1).strip()
            value = mod_match.group(2)
            modification_list.append(f"{factor}:{value}")
    
    if modification_list:
        result['modification_factors'] = '; '.join(modification_list)
    
    # 4. 관련판례 찾기 (대법원 YYYY.MM.DD 패턴)
    case_pattern = r'대법원\s*(\d{4}\.\d{1,2}\.\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)'
    case_matches = re.findall(case_pattern, text)
    if case_matches:
        result['related_cases'] = '; '.join(case_matches)
    
    # 데이터가 있으면 반환, 없으면 None
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    return None

# 실행 예시
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # Step 1 실행
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    # 결과 확인
    if data_standards:
        df_temp = pd.DataFrame(data_standards)
        print("\n📊 추출된 데이터 미리보기:")
        print(df_temp[['accident_code', 'accident_name', 'basic_fault_ratio']].head())
        
        # 임시 저장
        df_temp.to_csv("temp_step1_standards.csv", index=False, encoding='utf-8-sig')
        print("💾 Step1 결과가 temp_step1_standards.csv로 저장되었습니다.")
    else:
        print("❌ 추출된 데이터가 없습니다. 패턴을 확인해주세요.")

📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 페이지 48 처리 중...
   📄 페

   📄 페이지 381 처리 중...
   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 처리 중...
   📄 페이지 425 처리 중...
   📄 페이지 426 처리 중...
   📄 페이지 427 처리 중...
   📄 페이지 428 

# 1. 수정보완(1)

In [3]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출 - 개선된 버전
    - 실제 사고유형 상세 페이지만 추출 (목차 제외)
    - 기본과실비율, 수정요소, 관련판례 정확히 추출
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
            
            # 목차나 설명 페이지 제외 - 실제 사고유형 상세 페이지만 처리
            if is_detail_page(text):
                page_data = extract_page_data_standards_improved(text, page_num + 1)
                if page_data:
                    extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def is_detail_page(text):
    """
    실제 사고유형 상세 페이지인지 판단
    - 목차, 설명, 서론 페이지 제외
    - 기본과실비율, 수정요소가 있는 실제 상세 페이지만 선택
    """
    # 제외할 페이지 키워드
    exclude_keywords = [
        '목차', '서론', '적용 범위', '용어 정의',
        '제3편', '제1장', '제2장', '제3장',
        '............', '.........',  # 목차의 점선들
        'Page', '002', '003', '004'  # 페이지 번호만 있는 경우
    ]
    
    # 포함되어야 할 키워드 (실제 상세 페이지)
    include_keywords = [
        '기본과실비율', '수정요소', '과실비율',
        '①', '②', '③', '④', '⑤',  # 수정요소 번호
        '대법원', '판결',  # 관련판례
        '%'  # 비율 표시
    ]
    
    # 제외 키워드가 많으면 목차 페이지
    exclude_count = sum(1 for keyword in exclude_keywords if keyword in text)
    if exclude_count >= 3:
        return False
    
    # 포함 키워드가 있으면 상세 페이지
    include_count = sum(1 for keyword in include_keywords if keyword in text)
    return include_count >= 2

def extract_page_data_standards_improved(text, page_num):
    """
    개선된 과실비율인정기준 PDF 데이터 추출
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:500]  # 디버깅용
    }
    
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    
    # 1. 사고유형코드 찾기 (개선된 패턴)
    accident_code_found = False
    for i, line in enumerate(clean_lines):
        # [보1], [차1] 같은 패턴으로 둘러싸인 코드 우선 찾기
        bracket_match = re.search(r'\[([보차이]\d+)\]', line)
        if bracket_match:
            result['accident_code'] = bracket_match.group(1)
            accident_code_found = True
            
            # 같은 줄 또는 근처에서 사고유형명 찾기
            name_candidates = [line.replace(bracket_match.group(0), '').strip()]
            
            # 앞뒤 줄도 확인
            if i > 0:
                name_candidates.append(clean_lines[i-1])
            if i < len(clean_lines) - 1:
                name_candidates.append(clean_lines[i+1])
            
            # 가장 적절한 사고유형명 선택
            for candidate in name_candidates:
                if candidate and not re.search(r'[\d\.]{3,}|페이지|Page', candidate):
                    # 불필요한 부분 제거
                    clean_name = re.sub(r'\[.*?\]|\.{3,}|\d{3,}', '', candidate).strip()
                    if len(clean_name) > 5:  # 너무 짧은 건 제외
                        result['accident_name'] = clean_name[:100]  # 길이 제한
                        break
            break
    
    # 단순 패턴도 시도 (대비책)
    if not accident_code_found:
        for line in clean_lines:
            simple_match = re.search(r'^([보차이]\d+)', line)
            if simple_match:
                result['accident_code'] = simple_match.group(1)
                # 같은 줄에서 사고유형명 추출
                remaining = line[simple_match.end():].strip()
                if remaining:
                    result['accident_name'] = remaining[:100]
                break
    
    # 2. 기본과실비율 찾기 (개선된 패턴)
    ratio_patterns = [
        r'기본과실비율.*?(\d+)',  # "기본과실비율 70"
        r'보행자.*?기본.*?과실비율.*?(\d+)',  # "보행자 기본 과실비율 70"
        r'과실비율.*?(\d{1,2})(?:\s*%)?',  # "과실비율 70%" 또는 "과실비율 70"
        r'(\d{1,2})\s*%',  # 단순히 "70%"
        r'기본.*?(\d{1,2})',  # "기본 70"
    ]
    
    text_joined = ' '.join(clean_lines)
    for pattern in ratio_patterns:
        ratio_match = re.search(pattern, text_joined, re.IGNORECASE)
        if ratio_match:
            ratio_value = int(ratio_match.group(1))
            if 0 <= ratio_value <= 100:  # 유효한 비율만
                result['basic_fault_ratio'] = str(ratio_value)
                break
    
    # 3. 수정요소 찾기 (개선된 패턴)
    modification_list = []
    modification_patterns = [
        r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩]+?)(?:[+\-]?\d+|비례)',
        r'(\d+)\.\s*([^0-9\n]+?)(?:[+\-]?\d+|비례)',
        r'([가나다라마바사아자차카타파하])\s*([^가나다라마바사아자차카타파하]+?)(?:[+\-]?\d+|비례)',
    ]
    
    for line in clean_lines:
        for pattern in modification_patterns:
            matches = re.finditer(pattern, line)
            for match in matches:
                if len(match.groups()) >= 2:
                    factor_desc = match.group(2).strip()
                    # 점수 추출
                    score_match = re.search(r'([+\-]?\d+)', line[match.end()-10:match.end()+10])
                    score = score_match.group(1) if score_match else ''
                    
                    if factor_desc and len(factor_desc) > 3:
                        modification_list.append(f"{factor_desc}:{score}")
    
    # 중복 제거 및 정리
    seen = set()
    unique_modifications = []
    for mod in modification_list:
        if mod not in seen and len(mod) > 5:
            seen.add(mod)
            unique_modifications.append(mod)
    
    result['modification_factors'] = '; '.join(unique_modifications[:10])  # 최대 10개
    
    # 4. 관련판례 찾기 (개선된 패턴)
    case_patterns = [
        r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
        r'대법원\s*(\d{4}-\d+-\d+)',  # 간단한 형태
        r'판결\s*(\d{4}\.\d{1,2}\.\d{1,2})',
        r'(\d{4}년.*?판결)',
    ]
    
    case_matches = []
    for pattern in case_patterns:
        matches = re.findall(pattern, text_joined)
        case_matches.extend(matches)
    
    if case_matches:
        # 중복 제거하고 최대 3개까지
        unique_cases = list(dict.fromkeys(case_matches))[:3]
        result['related_cases'] = '; '.join(unique_cases)
    
    # 5. 데이터 유효성 검사
    # 최소한 사고코드나 기본과실비율 중 하나는 있어야 유효한 데이터
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    
    return None

# 실행 및 검증 함수
def validate_extracted_data(data_list):
    """추출된 데이터 품질 검증"""
    if not data_list:
        return
    
    print(f"\n📊 데이터 품질 검증 보고서")
    print("="*50)
    
    total = len(data_list)
    has_code = sum(1 for d in data_list if d['accident_code'])
    has_name = sum(1 for d in data_list if d['accident_name'])
    has_ratio = sum(1 for d in data_list if d['basic_fault_ratio'])
    has_modification = sum(1 for d in data_list if d['modification_factors'])
    has_cases = sum(1 for d in data_list if d['related_cases'])
    
    print(f"📈 전체 데이터: {total}건")
    print(f"   - 사고코드 있음: {has_code}건 ({has_code/total*100:.1f}%)")
    print(f"   - 사고유형명 있음: {has_name}건 ({has_name/total*100:.1f}%)")
    print(f"   - 기본과실비율 있음: {has_ratio}건 ({has_ratio/total*100:.1f}%)")
    print(f"   - 수정요소 있음: {has_modification}건 ({has_modification/total*100:.1f}%)")
    print(f"   - 관련판례 있음: {has_cases}건 ({has_cases/total*100:.1f}%)")
    
    # 상위 5개 샘플 출력
    print(f"\n📋 추출된 데이터 샘플:")
    for i, item in enumerate(data_list[:5]):
        print(f"\n{i+1}. 사고코드: {item['accident_code']}")
        print(f"   사고유형명: {item['accident_name'][:50]}...")
        print(f"   기본과실비율: {item['basic_fault_ratio']}")
        print(f"   수정요소: {item['modification_factors'][:50]}...")
        print(f"   관련판례: {item['related_cases'][:50]}...")

# 실행 예시
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # Step 1 실행
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    # 데이터 품질 검증
    validate_extracted_data(data_standards)
    
    # 결과 저장
    if data_standards:
        df_temp = pd.DataFrame(data_standards)
        df_temp.to_csv("temp_step1_standards_improved.csv", index=False, encoding='utf-8-sig')
        print(f"\n💾 개선된 Step1 결과가 temp_step1_standards_improved.csv로 저장되었습니다.")
    else:
        print("❌ 추출된 데이터가 없습니다. 패턴을 다시 확인해주세요.")

📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 페이지 48 처리 중...
   📄 페

   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 처리 중...
   📄 페이지 425 처리 중...
   📄 페이지 426 처리 중...
   📄 페이지 427 처리 중...
   📄 페이지 428 처리 중...
   📄 페이지 429 

# 2. 수정개선보완(2)

In [5]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출 - 최종 개선 버전
    - 사고유형명, 관련판례 추출 대폭 개선
    - 페이지별 상세 분석으로 정확도 극대화
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
            
            # 실제 사고유형 상세 페이지만 처리
            if is_accident_detail_page(text):
                page_data = extract_page_data_final(text, page_num + 1)
                if page_data:
                    extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def is_accident_detail_page(text):
    """
    사고유형 상세 페이지 판별 - 더 정교한 필터링
    """
    # 반드시 제외할 페이지들
    exclude_strong = [
        '목차', '서론', '적용 범위', '용어 정의', 'Page',
        '제3편', '제1장', '제2장', '제3장',
        '..........................'  # 목차 점선
    ]
    
    # 제외할 가능성이 높은 페이지
    exclude_weak = [
        '002', '003', '004', '005',  # 페이지 번호만 있는 경우
        '자동차사고 과실비율 인정기준',
        '과실비율 적용기준'
    ]
    
    # 반드시 포함되어야 할 요소들
    must_have = [
        r'[보차이]\d+',  # 사고코드
        r'\d{1,2}(?:\s*%)?',  # 비율
    ]
    
    # 포함되면 좋은 요소들
    good_to_have = [
        '기본과실비율', '기본 과실비율', '수정요소',
        '①', '②', '③', '④', '⑤',
        '대법원', '판결', '사고 상황',
        '+', '-', '%'
    ]
    
    # 강력 제외 조건
    for exclude in exclude_strong:
        if exclude in text and len(text) < 1000:  # 짧은 텍스트에서 발견되면 제외
            return False
    
    # 필수 조건 확인
    must_count = sum(1 for pattern in must_have if re.search(pattern, text))
    if must_count < 1:
        return False
    
    # 선호 조건 확인
    good_count = sum(1 for keyword in good_to_have if keyword in text)
    weak_count = sum(1 for keyword in exclude_weak if keyword in text)
    
    # 점수 기반 판별
    score = good_count * 2 - weak_count
    return score >= 3

def extract_page_data_final(text, page_num):
    """
    최종 개선된 데이터 추출 함수
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:300]  # 디버깅용
    }
    
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    text_joined = ' '.join(clean_lines)
    
    # === 1. 사고유형코드 추출 (최우선) ===
    accident_code = extract_accident_code(clean_lines, text_joined)
    if accident_code:
        result['accident_code'] = accident_code
    
    # === 2. 사고유형명 추출 (대폭 개선) ===
    accident_name = extract_accident_name(clean_lines, text_joined, accident_code)
    if accident_name:
        result['accident_name'] = accident_name
    
    # === 3. 기본과실비율 추출 ===
    basic_ratio = extract_basic_fault_ratio(clean_lines, text_joined)
    if basic_ratio:
        result['basic_fault_ratio'] = basic_ratio
    
    # === 4. 수정요소 추출 ===
    modifications = extract_modification_factors(clean_lines, text_joined)
    if modifications:
        result['modification_factors'] = modifications
    
    # === 5. 관련판례 추출 (개선) ===
    cases = extract_related_cases(clean_lines, text_joined)
    if cases:
        result['related_cases'] = cases
    
    # 최소 조건: 사고코드 또는 기본과실비율이 있어야 유효
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    
    return None

def extract_accident_code(lines, text_joined):
    """사고유형코드 추출"""
    # 패턴 1: [보1], [차1] 형태 (최우선)
    bracket_match = re.search(r'\[([보차이]\d+)\]', text_joined)
    if bracket_match:
        return bracket_match.group(1)
    
    # 패턴 2: 보1), 차1) 형태 
    paren_match = re.search(r'([보차이]\d+)\)', text_joined)
    if paren_match:
        return paren_match.group(1)
    
    # 패턴 3: 줄 시작에 보1, 차1 형태
    for line in lines:
        if re.match(r'^([보차이]\d+)', line):
            return re.match(r'^([보차이]\d+)', line).group(1)
    
    # 패턴 4: 단독으로 존재하는 코드
    standalone_match = re.search(r'\b([보차이]\d+)\b', text_joined)
    if standalone_match:
        return standalone_match.group(1)
    
    return None

def extract_accident_name(lines, text_joined, accident_code):
    """사고유형명 추출 - 대폭 개선"""
    if not accident_code:
        return None
    
    candidates = []
    
    # === 방법 1: 코드 근처 텍스트에서 추출 ===
    code_patterns = [
        rf'\[{accident_code}\]([^[\n]+)',  # [보1] 직진 대 직진
        rf'{accident_code}\)([^)\n]+)',    # 보1) 직진 대 직진  
        rf'{accident_code}([^0-9\n]+)',    # 보1 직진 대 직진
    ]
    
    for pattern in code_patterns:
        match = re.search(pattern, text_joined)
        if match:
            name_candidate = match.group(1).strip()
            # 정제
            name_candidate = re.sub(r'[\.\-\s]{3,}.*', '', name_candidate)  # 점선 제거
            name_candidate = re.sub(r'\d{3,}.*', '', name_candidate)       # 페이지번호 제거
            if 10 <= len(name_candidate) <= 100:  # 적절한 길이
                candidates.append(name_candidate)
    
    # === 방법 2: 사고 설명 패턴 찾기 ===
    accident_patterns = [
        r'(.*?(?:직진|좌회전|우회전|후진|주차).*?(?:대|사고).*?)',
        r'(.*?(?:보행자|자동차|이륜차).*?(?:횡단|충돌|접촉).*?)',
        r'(.*?(?:신호|교차로|횡단보도).*?(?:통과|진입).*?)',
        r'(\([^)]+\)\s*\([^)]+\))',  # (신호등 있음) (직진)
    ]
    
    for line in lines:
        for pattern in accident_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                name_candidate = match.group(1).strip()
                # 불필요한 부분 제거
                name_candidate = re.sub(r'^\d+[\.\)]\s*', '', name_candidate)  # 번호 제거
                name_candidate = re.sub(r'\[.*?\]', '', name_candidate)        # 괄호 제거
                name_candidate = re.sub(r'\.{3,}.*', '', name_candidate)       # 점선 제거
                if 8 <= len(name_candidate) <= 80:
                    candidates.append(name_candidate)
    
    # === 방법 3: 구조화된 텍스트에서 추출 ===
    structure_patterns = [
        r'사고\s*상황[:\s]*([^:\n]+)',
        r'사고\s*유형[:\s]*([^:\n]+)', 
        r'적용\s*기준[:\s]*([^:\n]+)',
    ]
    
    for pattern in structure_patterns:
        match = re.search(pattern, text_joined, re.IGNORECASE)
        if match:
            name_candidate = match.group(1).strip()
            if 10 <= len(name_candidate) <= 100:
                candidates.append(name_candidate)
    
    # === 최적 후보 선택 ===
    if candidates:
        # 중복 제거
        unique_candidates = list(dict.fromkeys(candidates))
        
        # 점수 매기기
        scored_candidates = []
        for candidate in unique_candidates:
            score = 0
            # 키워드 포함 점수
            keywords = ['직진', '좌회전', '우회전', '보행자', '자동차', '교차로', '횡단', '신호', '사고']
            score += sum(3 for keyword in keywords if keyword in candidate)
            
            # 길이 점수 (적당한 길이 선호)
            if 15 <= len(candidate) <= 50:
                score += 5
            elif 10 <= len(candidate) <= 60:
                score += 3
            
            # 특수문자 패널티
            if re.search(r'[\.]{3,}|\d{3,}', candidate):
                score -= 5
            
            scored_candidates.append((candidate, score))
        
        # 가장 높은 점수의 후보 선택
        if scored_candidates:
            best_candidate = max(scored_candidates, key=lambda x: x[1])
            if best_candidate[1] > 0:  # 최소 점수 이상
                return best_candidate[0]
    
    return None

def extract_basic_fault_ratio(lines, text_joined):
    """기본과실비율 추출"""
    patterns = [
        r'기본\s*과실\s*비율[:\s]*(\d{1,2})',
        r'기본[:\s]*(\d{1,2})\s*%?',
        r'과실\s*비율[:\s]*(\d{1,2})',
        r'(\d{1,2})\s*%',
        r'비율[:\s]*(\d{1,2})',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text_joined, re.IGNORECASE)
        if match:
            ratio = int(match.group(1))
            if 0 <= ratio <= 100:
                return str(ratio)
    
    return None

def extract_modification_factors(lines, text_joined):
    """수정요소 추출"""
    factors = []
    
    # 패턴 1: ①, ②, ③ 형태
    circle_patterns = r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(circle_patterns, text_joined)
    for match in matches:
        desc = match.group(1).strip()
        score = match.group(2) if match.group(2) else ''
        if len(desc) > 3:
            factors.append(f"{desc}:{score}")
    
    # 패턴 2: 가. 나. 다. 형태
    korean_patterns = r'[가나다라마바사아자차카타파하]\.\s*([^가나다라마바사아자차카타파하\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(korean_patterns, text_joined)
    for match in matches:
        desc = match.group(1).strip()
        score = match.group(2) if match.group(2) else ''
        if len(desc) > 3:
            factors.append(f"{desc}:{score}")
    
    # 패턴 3: 1), 2), 3) 형태
    number_patterns = r'(\d+)\)\s*([^\d\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(number_patterns, text_joined)
    for match in matches:
        desc = match.group(2).strip()
        score = match.group(3) if match.group(3) else ''
        if len(desc) > 3 and not re.search(r'페이지|Page', desc):
            factors.append(f"{desc}:{score}")
    
    # 중복 제거 및 정리
    unique_factors = []
    seen = set()
    for factor in factors:
        if factor not in seen and len(factor) > 5:
            seen.add(factor)
            unique_factors.append(factor)
    
    return '; '.join(unique_factors[:8]) if unique_factors else None

def extract_related_cases(lines, text_joined):
    """관련판례 추출 - 개선"""
    cases = []
    
    # 패턴들 (더 다양하게)
    patterns = [
        r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
        r'대법원\s*(\d{4}-\d+-\d+)',
        r'대법원\s*(\d{4}\.?\d{1,2}\.?\d{1,2})',  
        r'(\d{4}년\s*\d{1,2}월\s*\d{1,2}일\s*판결)',
        r'판결\s*(\d{4}\.\d{1,2}\.\d{1,2})',
        r'선고\s*(\d+[가-힣]\d+)',
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text_joined)
        cases.extend(matches)
    
    # 판례번호 패턴도 찾기
    case_num_patterns = [
        r'(\d+[가-힣]\d+)',  # 2019다12345
        r'(\d{4}[가-힣]\d+)', # 2019가단12345  
    ]
    
    for pattern in case_num_patterns:
        matches = re.findall(pattern, text_joined)
        for match in matches:
            if not any(match in case for case in cases):  # 중복 방지
                cases.append(match)
    
    # 정리 및 중복 제거
    unique_cases = list(dict.fromkeys(cases))[:5]  # 최대 5개
    return '; '.join(unique_cases) if unique_cases else None

# 검증 및 실행 함수
def validate_and_save_data(data_list, filename):
    """데이터 검증 및 저장"""
    if not data_list:
        print("❌ 추출된 데이터가 없습니다.")
        return
    
    df = pd.DataFrame(data_list)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    # 상세 분석
    total = len(data_list)
    metrics = {
        'accident_code': sum(1 for d in data_list if d['accident_code']),
        'accident_name': sum(1 for d in data_list if d['accident_name']),
        'basic_fault_ratio': sum(1 for d in data_list if d['basic_fault_ratio']),
        'modification_factors': sum(1 for d in data_list if d['modification_factors']),
        'related_cases': sum(1 for d in data_list if d['related_cases']),
    }
    
    print(f"\n📊 최종 데이터 품질 보고서")
    print("="*50)
    print(f"전체 데이터: {total}건")
    for field, count in metrics.items():
        print(f"{field}: {count}건 ({count/total*100:.1f}%)")
    
    # 베스트 샘플 출력
    print(f"\n🏆 추출 품질이 좋은 상위 5개 샘플:")
    
    # 품질 점수 계산
    scored_data = []
    for item in data_list:
        score = 0
        if item['accident_code']: score += 20
        if item['accident_name']: score += 25
        if item['basic_fault_ratio']: score += 20
        if item['modification_factors']: score += 20
        if item['related_cases']: score += 15
        scored_data.append((item, score))
    
    # 상위 5개 출력
    top_samples = sorted(scored_data, key=lambda x: x[1], reverse=True)[:5]
    for i, (item, score) in enumerate(top_samples):
        print(f"\n{i+1}. [품질점수: {score}/100]")
        print(f"   코드: {item['accident_code']}")
        print(f"   사고명: {item['accident_name'][:60]}...")
        print(f"   기본비율: {item['basic_fault_ratio']}")
        print(f"   수정요소: {item['modification_factors'][:50]}...")
        print(f"   관련판례: {item['related_cases'][:50]}...")

# 실행
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    print("🚀 Step 1 최종 개선 버전 실행")
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    validate_and_save_data(data_standards, "temp_step1_standards_final.csv")
    print(f"\n💾 최종 결과가 temp_step1_standards_final.csv로 저장되었습니다.")

🚀 Step 1 최종 개선 버전 실행
📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 

   📄 페이지 377 처리 중...
   📄 페이지 378 처리 중...
   📄 페이지 379 처리 중...
   📄 페이지 380 처리 중...
   📄 페이지 381 처리 중...
   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 

# 3. 수정개선보완(EasyOCR)

In [1]:
import pdfplumber
import pandas as pd
import re
import os
import easyocr
import numpy as np
from PIL import Image
import time

class EnhancedFaultExtractor:
    def __init__(self, use_gpu=True):
        """
        EasyOCR 우선 과실비율 PDF 추출기
        """
        print("🚀 EasyOCR 기반 과실비율 추출기 초기화 중...")
        
        # EasyOCR 리더 초기화 (한국어 + 영어)
        try:
            self.reader = easyocr.Reader(['ko', 'en'], gpu=use_gpu)
            print(f"✅ EasyOCR 초기화 완료 (GPU: {use_gpu})")
        except Exception as e:
            print(f"⚠️  GPU 초기화 실패, CPU 모드로 전환: {e}")
            self.reader = easyocr.Reader(['ko', 'en'], gpu=False)
        
        self.extraction_stats = {
            'total_pages': 0,
            'ocr_success': 0,
            'pdfplumber_success': 0,
            'hybrid_success': 0,
            'extraction_method': []
        }

    def extract_fault_standards_pdf(self, pdf_path, max_pages=None):
        """
        EasyOCR 우선 전략으로 PDF 데이터 추출
        """
        print(f"📖 EasyOCR 우선 추출 시작: {os.path.basename(pdf_path)}")
        
        extracted_data = []
        
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if max_pages:
                total_pages = min(total_pages, max_pages)
            
            self.extraction_stats['total_pages'] = total_pages
            
            for page_num in range(total_pages):
                page = pdf.pages[page_num]
                print(f"   📄 페이지 {page_num + 1}/{total_pages} 처리 중...")
                
                # 1단계: EasyOCR 우선 시도
                text_ocr = self.extract_text_with_easyocr(page, page_num + 1)
                
                # 2단계: pdfplumber 백업 추출
                text_pdf = self.extract_text_with_pdfplumber(page)
                
                # 3단계: 최적 텍스트 선택
                best_text, method = self.select_best_text(text_ocr, text_pdf, page_num + 1)
                
                # 4단계: 데이터 추출 시도
                if best_text and self.is_accident_detail_page(best_text):
                    page_data = self.extract_page_data_enhanced(best_text, page_num + 1, method)
                    if page_data:
                        extracted_data.append(page_data)
                        print(f"      ✅ 데이터 추출 성공 ({method})")
                    else:
                        print(f"      ⚠️  텍스트는 있으나 데이터 추출 실패")
        
        self.print_extraction_stats()
        print(f"✅ 추출 완료: 총 {len(extracted_data)}개 사례")
        return extracted_data

    def extract_text_with_easyocr(self, page, page_num):
        """EasyOCR로 텍스트 추출"""
        try:
            start_time = time.time()
            
            # 고해상도 이미지 변환
            pil_image = page.to_image(resolution=300).original
            image_array = np.array(pil_image)
            
            # EasyOCR 실행
            results = self.reader.readtext(image_array)
            
            # 결과를 텍스트로 조합 (위치 정보 고려)
            if results:
                # 위치 기반 정렬 (위에서 아래로, 왼쪽에서 오른쪽으로)
                sorted_results = sorted(results, key=lambda x: (x[0][0][1], x[0][0][0]))
                extracted_text = '\n'.join([result[1] for result in sorted_results])
                
                elapsed = time.time() - start_time
                print(f"      🔍 EasyOCR: {len(extracted_text)}자 추출 ({elapsed:.1f}초)")
                self.extraction_stats['ocr_success'] += 1
                return extracted_text
            else:
                print(f"      ❌ EasyOCR: 텍스트 없음")
                return ""
                
        except Exception as e:
            print(f"      ❌ EasyOCR 오류: {e}")
            return ""

    def extract_text_with_pdfplumber(self, page):
        """pdfplumber로 텍스트 추출 (백업용)"""
        try:
            text = page.extract_text()
            if text:
                print(f"      📄 pdfplumber: {len(text)}자 추출")
                self.extraction_stats['pdfplumber_success'] += 1
                return text
            else:
                print(f"      ❌ pdfplumber: 텍스트 없음")
                return ""
        except Exception as e:
            print(f"      ❌ pdfplumber 오류: {e}")
            return ""

    def select_best_text(self, text_ocr, text_pdf, page_num):
        """최적의 텍스트 선택 로직"""
        
        # 품질 점수 계산
        score_ocr = self.calculate_text_quality(text_ocr, "OCR")
        score_pdf = self.calculate_text_quality(text_pdf, "PDF")
        
        print(f"      📊 품질점수 - OCR: {score_ocr}, PDF: {score_pdf}")
        
        # 선택 로직
        if score_ocr > score_pdf * 1.3:  # OCR이 확실히 우수
            self.extraction_stats['extraction_method'].append('OCR')
            return text_ocr, 'EasyOCR'
        elif score_pdf > score_ocr * 1.3:  # PDF가 확실히 우수
            self.extraction_stats['extraction_method'].append('PDF')
            return text_pdf, 'pdfplumber'
        elif score_ocr > 0 and score_pdf > 0:  # 둘 다 있으면 하이브리드
            hybrid_text = self.create_hybrid_text(text_ocr, text_pdf)
            self.extraction_stats['hybrid_success'] += 1
            self.extraction_stats['extraction_method'].append('Hybrid')
            return hybrid_text, 'Hybrid'
        elif score_ocr > score_pdf:  # OCR 우선
            self.extraction_stats['extraction_method'].append('OCR')
            return text_ocr, 'EasyOCR'
        else:  # PDF 우선
            self.extraction_stats['extraction_method'].append('PDF')
            return text_pdf, 'pdfplumber'

    def calculate_text_quality(self, text, method_name):
        """텍스트 품질 점수 계산"""
        if not text:
            return 0
        
        score = 0
        
        # 기본 점수: 텍스트 길이
        score += min(len(text) / 100, 10)  # 최대 10점
        
        # 핵심 키워드 점수
        key_patterns = [
            (r'[보차이]\d+', 15),        # 사고코드 (15점)
            (r'기본.*과실.*비율', 10),      # 기본과실비율 (10점)
            (r'\d{1,2}\s*%', 8),         # 비율 표시 (8점)
            (r'[①②③④⑤⑥⑦⑧⑨⑩]', 5),   # 번호 매김 (5점)
            (r'수정.*요소', 5),           # 수정요소 (5점)
            (r'대법원.*판결', 8),         # 판례 (8점)
            (r'직진|좌회전|우회전', 3),    # 사고유형 (3점)
            (r'교차로|횡단보도', 3),       # 장소 (3점)
        ]
        
        for pattern, points in key_patterns:
            matches = len(re.findall(pattern, text))
            score += min(matches * points, points * 2)  # 중복 보너스 제한
        
        # 한국어 비율 점수 (OCR에 유리)
        korean_chars = len(re.findall(r'[가-힣]', text))
        total_chars = len(text)
        if total_chars > 0:
            korean_ratio = korean_chars / total_chars
            if method_name == "OCR":
                score += korean_ratio * 10  # OCR은 한국어 비율이 높을수록 좋음
        
        # 구조화 점수
        lines = text.split('\n')
        non_empty_lines = [line for line in lines if line.strip()]
        if len(non_empty_lines) > 5:
            score += 5  # 충분한 구조화
        
        # 노이즈 패널티
        if re.search(r'[^\w\s가-힣①-⑩\[\]\(\)\.,%\-\+]', text):
            score -= 3  # 이상한 문자 패널티
        
        return max(score, 0)

    def create_hybrid_text(self, text_ocr, text_pdf):
        """OCR과 PDF 텍스트를 결합하여 최적화"""
        
        # 방법 1: 키워드 기반 선택적 결합
        key_sections = {}
        
        # OCR에서 강점이 있는 부분 추출
        ocr_strengths = [
            (r'[보차이]\d+.*?(?=\n|$)', 'accident_codes'),
            (r'.*?[①②③④⑤⑥⑦⑧⑨⑩].*?(?=\n|$)', 'numbered_items'),
            (r'.*?기본.*과실.*비율.*?(?=\n|$)', 'fault_ratios'),
        ]
        
        for pattern, section in ocr_strengths:
            matches = re.findall(pattern, text_ocr, re.MULTILINE)
            if matches:
                key_sections[section] = matches
        
        # PDF에서 강점이 있는 부분 추출
        pdf_strengths = [
            (r'대법원.*?판결.*?(?=\n|$)', 'court_cases'),
            (r'.*?수정.*요소.*?(?=\n|$)', 'modification_factors'),
        ]
        
        for pattern, section in pdf_strengths:
            matches = re.findall(pattern, text_pdf, re.MULTILINE)
            if matches and section not in key_sections:
                key_sections[section] = matches
        
        # 결합된 텍스트 생성
        hybrid_parts = []
        
        # 주요 섹션들을 순서대로 추가
        section_order = ['accident_codes', 'fault_ratios', 'numbered_items', 'modification_factors', 'court_cases']
        
        for section in section_order:
            if section in key_sections:
                hybrid_parts.extend(key_sections[section])
        
        # 나머지 중요한 내용 추가 (OCR 우선)
        remaining_ocr = text_ocr
        for section, matches in key_sections.items():
            for match in matches:
                remaining_ocr = remaining_ocr.replace(match, '')
        
        # 정리된 텍스트 추가
        cleaned_remaining = '\n'.join([line.strip() for line in remaining_ocr.split('\n') 
                                     if line.strip() and len(line.strip()) > 5])
        if cleaned_remaining:
            hybrid_parts.append(cleaned_remaining)
        
        return '\n'.join(hybrid_parts)

    def is_accident_detail_page(self, text):
        """사고유형 상세 페이지 판별 - 기존 로직 유지"""
        exclude_strong = [
            '목차', '서론', '적용 범위', '용어 정의', 'Page',
            '제3편', '제1장', '제2장', '제3장',
            '..........................'
        ]
        
        exclude_weak = [
            '002', '003', '004', '005',
            '자동차사고 과실비율 인정기준',
            '과실비율 적용기준'
        ]
        
        must_have = [
            r'[보차이]\d+',
            r'\d{1,2}(?:\s*%)?',
        ]
        
        good_to_have = [
            '기본과실비율', '기본 과실비율', '수정요소',
            '①', '②', '③', '④', '⑤',
            '대법원', '판결', '사고 상황',
            '+', '-', '%'
        ]
        
        # 강력 제외 조건
        for exclude in exclude_strong:
            if exclude in text and len(text) < 1000:
                return False
        
        # 필수 조건 확인
        must_count = sum(1 for pattern in must_have if re.search(pattern, text))
        if must_count < 1:
            return False
        
        # 선호 조건 확인
        good_count = sum(1 for keyword in good_to_have if keyword in text)
        weak_count = sum(1 for keyword in exclude_weak if keyword in text)
        
        score = good_count * 2 - weak_count
        return score >= 3

    def extract_page_data_enhanced(self, text, page_num, extraction_method):
        """향상된 데이터 추출 - 기존 로직 활용"""
        result = {
            'source_pdf': '과실비율인정기준',
            'page_number': page_num,
            'extraction_method': extraction_method,
            'accident_code': '',
            'accident_name': '',
            'basic_fault_ratio': '',
            'modification_factors': '',
            'related_cases': '',
            'text_quality_score': self.calculate_text_quality(text, extraction_method),
            'raw_text': text[:300]
        }
        
        lines = text.split('\n')
        clean_lines = [line.strip() for line in lines if line.strip()]
        text_joined = ' '.join(clean_lines)
        
        # 기존 추출 로직들 활용
        result['accident_code'] = self.extract_accident_code(clean_lines, text_joined)
        result['accident_name'] = self.extract_accident_name(clean_lines, text_joined, result['accident_code'])
        result['basic_fault_ratio'] = self.extract_basic_fault_ratio(clean_lines, text_joined)
        result['modification_factors'] = self.extract_modification_factors(clean_lines, text_joined)
        result['related_cases'] = self.extract_related_cases(clean_lines, text_joined)
        
        # 최소 조건 확인
        if result['accident_code'] or result['basic_fault_ratio']:
            return result
        
        return None

    def extract_accident_code(self, lines, text_joined):
        """사고유형코드 추출 - 기존 로직"""
        bracket_match = re.search(r'\[([보차이]\d+)\]', text_joined)
        if bracket_match:
            return bracket_match.group(1)
        
        paren_match = re.search(r'([보차이]\d+)\)', text_joined)
        if paren_match:
            return paren_match.group(1)
        
        for line in lines:
            if re.match(r'^([보차이]\d+)', line):
                return re.match(r'^([보차이]\d+)', line).group(1)
        
        standalone_match = re.search(r'\b([보차이]\d+)\b', text_joined)
        if standalone_match:
            return standalone_match.group(1)
        
        return None

    def extract_accident_name(self, lines, text_joined, accident_code):
        """사고유형명 추출 - 기존 로직 활용"""
        if not accident_code:
            return None
        
        candidates = []
        
        # 코드 근처 텍스트 추출
        code_patterns = [
            rf'\[{accident_code}\]([^[\n]+)',
            rf'{accident_code}\)([^)\n]+)',
            rf'{accident_code}([^0-9\n]+)',
        ]
        
        for pattern in code_patterns:
            match = re.search(pattern, text_joined)
            if match:
                name_candidate = match.group(1).strip()
                name_candidate = re.sub(r'[\.\-\s]{3,}.*', '', name_candidate)
                name_candidate = re.sub(r'\d{3,}.*', '', name_candidate)
                if 10 <= len(name_candidate) <= 100:
                    candidates.append(name_candidate)
        
        if candidates:
            return candidates[0]  # 첫 번째 후보 반환
        
        return None

    def extract_basic_fault_ratio(self, lines, text_joined):
        """기본과실비율 추출 - 기존 로직"""
        patterns = [
            r'기본\s*과실\s*비율[:\s]*(\d{1,2})',
            r'기본[:\s]*(\d{1,2})\s*%?',
            r'과실\s*비율[:\s]*(\d{1,2})',
            r'(\d{1,2})\s*%',
            r'비율[:\s]*(\d{1,2})',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text_joined, re.IGNORECASE)
            if match:
                ratio = int(match.group(1))
                if 0 <= ratio <= 100:
                    return str(ratio)
        
        return None

    def extract_modification_factors(self, lines, text_joined):
        """수정요소 추출 - 기존 로직"""
        factors = []
        
        patterns = [
            r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩\n]+?)(?:([+\-]\d+)|비례)',
            r'[가나다라마바사아자차카타파하]\.\s*([^가나다라마바사아자차카타파하\n]+?)(?:([+\-]\d+)|비례)',
            r'(\d+)\)\s*([^\d\n]+?)(?:([+\-]\d+)|비례)'
        ]
        
        for pattern in patterns:
            matches = re.finditer(pattern, text_joined)
            for match in matches:
                if len(match.groups()) >= 2:
                    desc = match.group(1).strip() if match.group(1) else match.group(2).strip()
                    score = match.group(-1) if match.group(-1) else ''
                    if len(desc) > 3:
                        factors.append(f"{desc}:{score}")
        
        unique_factors = list(dict.fromkeys(factors))[:8]
        return '; '.join(unique_factors) if unique_factors else None

    def extract_related_cases(self, lines, text_joined):
        """관련판례 추출 - 기존 로직"""
        cases = []
        
        patterns = [
            r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
            r'대법원\s*(\d{4}-\d+-\d+)',
            r'대법원\s*(\d{4}\.?\d{1,2}\.?\d{1,2})',
            r'(\d{4}년\s*\d{1,2}월\s*\d{1,2}일\s*판결)',
            r'선고\s*(\d+[가-힣]\d+)',
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text_joined)
            cases.extend(matches)
        
        unique_cases = list(dict.fromkeys(cases))[:5]
        return '; '.join(unique_cases) if unique_cases else None

    def print_extraction_stats(self):
        """추출 통계 출력"""
        stats = self.extraction_stats
        print(f"\n📊 추출 방법별 통계:")
        print(f"   📄 전체 페이지: {stats['total_pages']}")
        print(f"   🔍 EasyOCR 성공: {stats['ocr_success']} ({stats['ocr_success']/stats['total_pages']*100:.1f}%)")
        print(f"   📋 pdfplumber 성공: {stats['pdfplumber_success']} ({stats['pdfplumber_success']/stats['total_pages']*100:.1f}%)")
        print(f"   🔄 하이브리드 생성: {stats['hybrid_success']}")
        
        # 최종 선택 방법 통계
        method_counts = {}
        for method in stats['extraction_method']:
            method_counts[method] = method_counts.get(method, 0) + 1
        
        print(f"   🏆 최종 선택된 방법:")
        for method, count in method_counts.items():
            print(f"      {method}: {count}회")

def main():
    """메인 실행 함수"""
    print("🚀 EasyOCR 우선 과실비율 PDF 추출기 시작")
    print("="*60)
    
    # PDF 파일 경로
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    # 추출기 초기화 및 실행
    extractor = EnhancedFaultExtractor(use_gpu=True)
    
    # 전체 추출 (테스트 시에는 max_pages=10 정도로 제한)
    data_list = extractor.extract_fault_standards_pdf(pdf_path, max_pages=20)
    
    # 결과 저장 및 분석
    if data_list:
        df = pd.DataFrame(data_list)
        filename = "easyocr_enhanced_fault_extraction.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✅ 결과 저장 완료: {filename}")
        print(f"📊 총 {len(data_list)}건의 데이터 추출")
        
        # 추출 방법별 성공률
        method_success = df.groupby('extraction_method').size()
        print(f"\n📈 추출 방법별 성공 데이터:")
        for method, count in method_success.items():
            print(f"   {method}: {count}건")
        
        # 품질 점수 분석
        avg_quality = df['text_quality_score'].mean()
        print(f"\n⭐ 평균 텍스트 품질 점수: {avg_quality:.1f}")
        
    else:
        print("❌ 추출된 데이터가 없습니다.")

if __name__ == "__main__":
    main()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


🚀 EasyOCR 우선 과실비율 PDF 추출기 시작
🚀 EasyOCR 기반 과실비율 추출기 초기화 중...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |███████-------------------------------------------| 15.7% Complete

Progress: |████████████████----------------------------------| 32.6% Complete

Progress: |████████████████████████--------------------------| 48.4% Complete

...

Progress: |█████████████████████████████████████████---------| 82.0% Complete

Progress: |█████████████████████████████████████████████████-| 98.2% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete✅ EasyOCR 초기화 완료 (GPU: True)
📖 EasyOCR 우선 추출 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1/20 처리 중...


C:\Users\kmj11\anaconda3\envs\gpudm\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


      🔍 EasyOCR: 30자 추출 (6.4초)
      📄 pdfplumber: 49자 추출
      📊 품질점수 - OCR: 6.633333333333333, PDF: 5.49
   📄 페이지 2/20 처리 중...
      🔍 EasyOCR: 390자 추출 (7.7초)
      📄 pdfplumber: 2441자 추출
      📊 품질점수 - OCR: 29.310256410256407, PDF: 30
   📄 페이지 3/20 처리 중...
      🔍 EasyOCR: 624자 추출 (8.3초)
      📄 pdfplumber: 2621자 추출
      📊 품질점수 - OCR: 59.93551282051282, PDF: 59
   📄 페이지 4/20 처리 중...
      🔍 EasyOCR: 816자 추출 (9.2초)
      📄 pdfplumber: 2826자 추출
      📊 품질점수 - OCR: 62.294803921568615, PDF: 62
   📄 페이지 5/20 처리 중...
      🔍 EasyOCR: 572자 추출 (8.0초)
      📄 pdfplumber: 2629자 추출
      📊 품질점수 - OCR: 59.545174825174826, PDF: 62
   📄 페이지 6/20 처리 중...
      🔍 EasyOCR: 671자 추출 (8.2초)
      📄 pdfplumber: 2867자 추출
      📊 품질점수 - OCR: 25.68764530551416, PDF: 24
   📄 페이지 7/20 처리 중...
      🔍 EasyOCR: 21자 추출 (5.4초)
      📄 pdfplumber: 21자 추출
      📊 품질점수 - OCR: 7.829047619047619, PDF: 0.21
   📄 페이지 8/20 처리 중...
      🔍 EasyOCR: 895자 추출 (8.8초)
      📄 pdfplumber: 902자 추출
      📊 품질점수 - OCR: 18.145530

In [2]:
import pdfplumber
import pandas as pd
import os
import easyocr
import numpy as np
from PIL import Image
import time
import re

class SimpleTextExtractor:
    def __init__(self, use_gpu=True):
        """
        단순한 텍스트 추출기 - 복잡한 필터링 없이 모든 텍스트 저장
        """
        print("🚀 단순 텍스트 추출기 초기화 중...")
        
        try:
            self.reader = easyocr.Reader(['ko', 'en'], gpu=use_gpu)
            print(f"✅ EasyOCR 초기화 완료 (GPU: {use_gpu})")
        except Exception as e:
            print(f"⚠️  GPU 초기화 실패, CPU 모드로 전환: {e}")
            self.reader = easyocr.Reader(['ko', 'en'], gpu=False)

    def extract_all_text(self, pdf_path, output_format='both', max_pages=None):
        """
        모든 페이지에서 텍스트 추출하여 저장
        
        Args:
            pdf_path: PDF 파일 경로
            output_format: 'csv', 'txt', 'both' 중 선택
            max_pages: 처리할 최대 페이지 수 (None이면 전체)
        """
        print(f"📖 전체 텍스트 추출 시작: {os.path.basename(pdf_path)}")
        
        results = []
        
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if max_pages:
                total_pages = min(total_pages, max_pages)
            
            print(f"📄 총 {total_pages}페이지 처리 예정")
            
            for page_num in range(total_pages):
                page = pdf.pages[page_num]
                print(f"   📄 페이지 {page_num + 1}/{total_pages} 처리 중...")
                
                # EasyOCR 추출
                text_ocr = self.extract_with_easyocr(page, page_num + 1)
                
                # pdfplumber 추출
                text_pdf = self.extract_with_pdfplumber(page)
                
                # 최적 텍스트 선택
                best_text, method = self.choose_better_text(text_ocr, text_pdf)
                
                # 결과 저장
                page_result = {
                    'page_number': page_num + 1,
                    'extraction_method': method,
                    'text_length': len(best_text),
                    'ocr_text': text_ocr,
                    'pdf_text': text_pdf,
                    'selected_text': best_text,
                    'has_korean': len(re.findall(r'[가-힣]', best_text)) > 0,
                    'has_numbers': len(re.findall(r'\d', best_text)) > 0,
                    'contains_keywords': self.check_keywords(best_text)
                }
                
                results.append(page_result)
                
                print(f"      ✅ 완료 - {method}: {len(best_text)}자")
        
        # 결과 저장
        base_filename = os.path.splitext(os.path.basename(pdf_path))[0]
        
        if output_format in ['csv', 'both']:
            self.save_to_csv(results, f"{base_filename}_extracted_text.csv")
        
        if output_format in ['txt', 'both']:
            self.save_to_txt(results, f"{base_filename}_extracted_text.txt")
        
        # 통계 출력
        self.print_statistics(results)
        
        return results

    def extract_with_easyocr(self, page, page_num):
        """EasyOCR로 텍스트 추출"""
        try:
            start_time = time.time()
            
            # 고해상도 이미지 변환
            pil_image = page.to_image(resolution=300).original
            image_array = np.array(pil_image)
            
            # EasyOCR 실행
            results = self.reader.readtext(image_array)
            
            if results:
                # 위치 기반 정렬 (위→아래, 왼쪽→오른쪽)
                sorted_results = sorted(results, key=lambda x: (x[0][0][1], x[0][0][0]))
                extracted_text = '\n'.join([result[1] for result in sorted_results])
                
                elapsed = time.time() - start_time
                print(f"      🔍 EasyOCR: {len(extracted_text)}자 ({elapsed:.1f}초)")
                return extracted_text
            else:
                print(f"      ❌ EasyOCR: 텍스트 없음")
                return ""
                
        except Exception as e:
            print(f"      ❌ EasyOCR 오류: {e}")
            return ""

    def extract_with_pdfplumber(self, page):
        """pdfplumber로 텍스트 추출"""
        try:
            text = page.extract_text()
            if text:
                print(f"      📄 pdfplumber: {len(text)}자")
                return text
            else:
                print(f"      ❌ pdfplumber: 텍스트 없음")
                return ""
        except Exception as e:
            print(f"      ❌ pdfplumber 오류: {e}")
            return ""

    def choose_better_text(self, text_ocr, text_pdf):
        """더 나은 텍스트 선택"""
        
        # 둘 다 비어있으면
        if not text_ocr and not text_pdf:
            return "", "None"
        
        # 하나만 있으면 그것 선택
        if not text_ocr:
            return text_pdf, "pdfplumber"
        if not text_pdf:
            return text_ocr, "EasyOCR"
        
        # 둘 다 있으면 품질 비교
        score_ocr = self.calculate_simple_score(text_ocr)
        score_pdf = self.calculate_simple_score(text_pdf)
        
        # 길이도 고려 (너무 짧으면 점수 감점)
        if len(text_ocr) < 50:
            score_ocr *= 0.5
        if len(text_pdf) < 50:
            score_pdf *= 0.5
        
        if score_ocr > score_pdf:
            return text_ocr, "EasyOCR"
        else:
            return text_pdf, "pdfplumber"

    def calculate_simple_score(self, text):
        """간단한 텍스트 품질 점수"""
        if not text:
            return 0
        
        score = 0
        
        # 기본 점수: 텍스트 길이
        score += len(text) / 100
        
        # 한국어 있으면 보너스
        korean_count = len(re.findall(r'[가-힣]', text))
        score += korean_count / 10
        
        # 숫자 있으면 보너스
        number_count = len(re.findall(r'\d', text))
        score += number_count / 20
        
        # 특수 키워드 보너스
        keywords = ['보1', '보2', '차1', '차2', '과실', '비율', '%', '대법원', '판결']
        for keyword in keywords:
            if keyword in text:
                score += 5
        
        return score

    def check_keywords(self, text):
        """주요 키워드 포함 여부 확인"""
        keywords = [
            '과실', '비율', '사고', '보행자', '자동차', '교차로', 
            '직진', '좌회전', '우회전', '신호', '횡단보도',
            '보1', '보2', '보3', '차1', '차2', '차3',
            '대법원', '판결', '①', '②', '③'
        ]
        
        found_keywords = [kw for kw in keywords if kw in text]
        return ', '.join(found_keywords) if found_keywords else 'None'

    def save_to_csv(self, results, filename):
        """CSV 파일로 저장"""
        print(f"\n💾 CSV 저장 중: {filename}")
        
        # DataFrame 생성
        df_data = []
        for result in results:
            df_data.append({
                'page_number': result['page_number'],
                'extraction_method': result['extraction_method'],
                'text_length': result['text_length'],
                'has_korean': result['has_korean'],
                'has_numbers': result['has_numbers'],
                'contains_keywords': result['contains_keywords'],
                'selected_text': result['selected_text'],
                'ocr_text': result['ocr_text'],
                'pdf_text': result['pdf_text']
            })
        
        df = pd.DataFrame(df_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ CSV 저장 완료: {filename}")

    def save_to_txt(self, results, filename):
        """TXT 파일로 저장 (읽기 편한 형태)"""
        print(f"\n💾 TXT 저장 중: {filename}")
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("PDF 텍스트 추출 결과\n")
            f.write("=" * 80 + "\n\n")
            
            for result in results:
                f.write(f"📄 페이지 {result['page_number']}\n")
                f.write(f"📊 추출방법: {result['extraction_method']}\n")
                f.write(f"📏 텍스트 길이: {result['text_length']}자\n")
                f.write(f"🔍 포함 키워드: {result['contains_keywords']}\n")
                f.write("-" * 60 + "\n")
                f.write(result['selected_text'])
                f.write("\n\n" + "=" * 80 + "\n\n")
        
        print(f"✅ TXT 저장 완료: {filename}")

    def print_statistics(self, results):
        """추출 통계 출력"""
        total_pages = len(results)
        
        # 추출 방법별 통계
        method_counts = {}
        for result in results:
            method = result['extraction_method']
            method_counts[method] = method_counts.get(method, 0) + 1
        
        # 텍스트 품질 통계
        text_lengths = [r['text_length'] for r in results]
        avg_length = sum(text_lengths) / len(text_lengths) if text_lengths else 0
        
        korean_pages = sum(1 for r in results if r['has_korean'])
        keyword_pages = sum(1 for r in results if r['contains_keywords'] != 'None')
        
        print(f"\n📊 최종 추출 통계")
        print("=" * 50)
        print(f"📄 총 페이지: {total_pages}")
        print(f"📝 평균 텍스트 길이: {avg_length:.0f}자")
        print(f"🇰🇷 한국어 포함 페이지: {korean_pages}페이지 ({korean_pages/total_pages*100:.1f}%)")
        print(f"🔍 키워드 포함 페이지: {keyword_pages}페이지 ({keyword_pages/total_pages*100:.1f}%)")
        
        print(f"\n🏆 추출 방법별 선택 횟수:")
        for method, count in method_counts.items():
            print(f"   {method}: {count}회 ({count/total_pages*100:.1f}%)")
        
        # 최고 품질 페이지 Top 5
        top_pages = sorted(results, key=lambda x: x['text_length'], reverse=True)[:5]
        print(f"\n🏅 텍스트 길이 상위 5페이지:")
        for i, page in enumerate(top_pages):
            keywords = page['contains_keywords'][:50] + "..." if len(page['contains_keywords']) > 50 else page['contains_keywords']
            print(f"   {i+1}. 페이지 {page['page_number']}: {page['text_length']}자 ({page['extraction_method']}) - {keywords}")

def main():
    """메인 실행 함수"""
    print("🚀 단순 텍스트 추출기 시작")
    print("="*60)
    
    # PDF 파일 경로
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    # 추출기 초기화
    extractor = SimpleTextExtractor(use_gpu=True)
    
    # 선택 메뉴
    print("\n📋 추출 옵션을 선택하세요:")
    print("1. 전체 페이지 추출 (CSV + TXT)")
    print("2. 처음 20페이지만 테스트 (CSV + TXT)")
    print("3. CSV만 저장")
    print("4. TXT만 저장")
    
    choice = input("\n선택 (1-4): ").strip()
    
    if choice == '1':
        results = extractor.extract_all_text(pdf_path, output_format='both')
    elif choice == '2':
        results = extractor.extract_all_text(pdf_path, output_format='both', max_pages=20)
    elif choice == '3':
        results = extractor.extract_all_text(pdf_path, output_format='csv')
    elif choice == '4':
        results = extractor.extract_all_text(pdf_path, output_format='txt')
    else:
        print("기본값으로 처음 20페이지 테스트를 실행합니다.")
        results = extractor.extract_all_text(pdf_path, output_format='both', max_pages=20)
    
    print(f"\n🎉 추출 완료!")
    print(f"📁 생성된 파일들을 확인하세요.")

if __name__ == "__main__":
    main()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


🚀 단순 텍스트 추출기 시작
🚀 단순 텍스트 추출기 초기화 중...
✅ EasyOCR 초기화 완료 (GPU: True)

📋 추출 옵션을 선택하세요:
1. 전체 페이지 추출 (CSV + TXT)
2. 처음 20페이지만 테스트 (CSV + TXT)
3. CSV만 저장
4. TXT만 저장

선택 (1-4): 1
📖 전체 텍스트 추출 시작: 231107_과실비율인정기준_온라인용.pdf
📄 총 600페이지 처리 예정
   📄 페이지 1/600 처리 중...
      🔍 EasyOCR: 30자 (5.5초)
      📄 pdfplumber: 49자
      ✅ 완료 - pdfplumber: 49자
   📄 페이지 2/600 처리 중...
      🔍 EasyOCR: 390자 (7.2초)
      📄 pdfplumber: 2441자
      ✅ 완료 - pdfplumber: 2441자
   📄 페이지 3/600 처리 중...
      🔍 EasyOCR: 624자 (7.9초)
      📄 pdfplumber: 2621자
      ✅ 완료 - pdfplumber: 2621자
   📄 페이지 4/600 처리 중...
      🔍 EasyOCR: 816자 (8.8초)
      📄 pdfplumber: 2826자
      ✅ 완료 - pdfplumber: 2826자
   📄 페이지 5/600 처리 중...
      🔍 EasyOCR: 572자 (7.8초)
      📄 pdfplumber: 2629자
      ✅ 완료 - pdfplumber: 2629자
   📄 페이지 6/600 처리 중...
      🔍 EasyOCR: 671자 (8.3초)
      📄 pdfplumber: 2867자
      ✅ 완료 - pdfplumber: 2867자
   📄 페이지 7/600 처리 중...
      🔍 EasyOCR: 21자 (5.4초)
      📄 pdfplumber: 21자
      ✅ 완료 - pdfplumber: 21자
   📄 페이지 8/600 처

      🔍 EasyOCR: 978자 (10.4초)
      📄 pdfplumber: 1056자
      ✅ 완료 - pdfplumber: 1056자
   📄 페이지 73/600 처리 중...
      🔍 EasyOCR: 695자 (8.7초)
      📄 pdfplumber: 739자
      ✅ 완료 - pdfplumber: 739자
   📄 페이지 74/600 처리 중...
      🔍 EasyOCR: 642자 (8.9초)
      📄 pdfplumber: 706자
      ✅ 완료 - pdfplumber: 706자
   📄 페이지 75/600 처리 중...
      🔍 EasyOCR: 1014자 (9.9초)
      📄 pdfplumber: 1052자
      ✅ 완료 - pdfplumber: 1052자
   📄 페이지 76/600 처리 중...
      🔍 EasyOCR: 1065자 (10.3초)
      📄 pdfplumber: 1130자
      ✅ 완료 - pdfplumber: 1130자
   📄 페이지 77/600 처리 중...
      🔍 EasyOCR: 854자 (9.7초)
      📄 pdfplumber: 924자
      ✅ 완료 - pdfplumber: 924자
   📄 페이지 78/600 처리 중...
      🔍 EasyOCR: 699자 (9.4초)
      📄 pdfplumber: 742자
      ✅ 완료 - pdfplumber: 742자
   📄 페이지 79/600 처리 중...
      🔍 EasyOCR: 1082자 (11.0초)
      📄 pdfplumber: 1120자
      ✅ 완료 - pdfplumber: 1120자
   📄 페이지 80/600 처리 중...
      🔍 EasyOCR: 1060자 (10.4초)
      📄 pdfplumber: 1130자
      ✅ 완료 - pdfplumber: 1130자
   📄 페이지 81/600 처리 중...
      🔍 Ea

      🔍 EasyOCR: 1051자 (10.1초)
      📄 pdfplumber: 1125자
      ✅ 완료 - pdfplumber: 1125자
   📄 페이지 146/600 처리 중...
      🔍 EasyOCR: 852자 (9.5초)
      📄 pdfplumber: 961자
      ✅ 완료 - pdfplumber: 961자
   📄 페이지 147/600 처리 중...
      🔍 EasyOCR: 466자 (7.8초)
      📄 pdfplumber: 524자
      ✅ 완료 - pdfplumber: 524자
   📄 페이지 148/600 처리 중...
      🔍 EasyOCR: 436자 (7.8초)
      📄 pdfplumber: 479자
      ✅ 완료 - pdfplumber: 479자
   📄 페이지 149/600 처리 중...
      🔍 EasyOCR: 910자 (9.6초)
      📄 pdfplumber: 980자
      ✅ 완료 - pdfplumber: 980자
   📄 페이지 150/600 처리 중...
      🔍 EasyOCR: 948자 (10.1초)
      📄 pdfplumber: 1029자
      ✅ 완료 - pdfplumber: 1029자
   📄 페이지 151/600 처리 중...
      🔍 EasyOCR: 673자 (8.5초)
      📄 pdfplumber: 736자
      ✅ 완료 - pdfplumber: 736자
   📄 페이지 152/600 처리 중...
      🔍 EasyOCR: 732자 (9.0초)
      📄 pdfplumber: 773자
      ✅ 완료 - pdfplumber: 773자
   📄 페이지 153/600 처리 중...
      🔍 EasyOCR: 929자 (9.5초)
      📄 pdfplumber: 988자
      ✅ 완료 - pdfplumber: 988자
   📄 페이지 154/600 처리 중...
      🔍 Easy

      🔍 EasyOCR: 477자 (7.9초)
      📄 pdfplumber: 513자
      ✅ 완료 - pdfplumber: 513자
   📄 페이지 219/600 처리 중...
      🔍 EasyOCR: 660자 (9.0초)
      📄 pdfplumber: 719자
      ✅ 완료 - pdfplumber: 719자
   📄 페이지 220/600 처리 중...
      🔍 EasyOCR: 1047자 (10.5초)
      📄 pdfplumber: 1122자
      ✅ 완료 - pdfplumber: 1122자
   📄 페이지 221/600 처리 중...
      🔍 EasyOCR: 1473자 (12.5초)
      📄 pdfplumber: 1538자
      ✅ 완료 - pdfplumber: 1538자
   📄 페이지 222/600 처리 중...
      🔍 EasyOCR: 619자 (8.4초)
      📄 pdfplumber: 669자
      ✅ 완료 - pdfplumber: 669자
   📄 페이지 223/600 처리 중...
      🔍 EasyOCR: 617자 (8.9초)
      📄 pdfplumber: 660자
      ✅ 완료 - pdfplumber: 660자
   📄 페이지 224/600 처리 중...
      🔍 EasyOCR: 952자 (10.1초)
      📄 pdfplumber: 1017자
      ✅ 완료 - pdfplumber: 1017자
   📄 페이지 225/600 처리 중...
      🔍 EasyOCR: 1085자 (11.1초)
      📄 pdfplumber: 1168자
      ✅ 완료 - pdfplumber: 1168자
   📄 페이지 226/600 처리 중...
      🔍 EasyOCR: 580자 (8.3초)
      📄 pdfplumber: 629자
      ✅ 완료 - pdfplumber: 629자
   📄 페이지 227/600 처리 중...
    

      🔍 EasyOCR: 598자 (8.7초)
      📄 pdfplumber: 665자
      ✅ 완료 - pdfplumber: 665자
   📄 페이지 292/600 처리 중...
      🔍 EasyOCR: 900자 (9.7초)
      📄 pdfplumber: 977자
      ✅ 완료 - pdfplumber: 977자
   📄 페이지 293/600 처리 중...
      🔍 EasyOCR: 783자 (9.1초)
      📄 pdfplumber: 823자
      ✅ 완료 - pdfplumber: 823자
   📄 페이지 294/600 처리 중...
      🔍 EasyOCR: 765자 (9.5초)
      📄 pdfplumber: 879자
      ✅ 완료 - pdfplumber: 879자
   📄 페이지 295/600 처리 중...
      🔍 EasyOCR: 945자 (9.9초)
      📄 pdfplumber: 1010자
      ✅ 완료 - pdfplumber: 1010자
   📄 페이지 296/600 처리 중...
      🔍 EasyOCR: 1058자 (10.6초)
      📄 pdfplumber: 1126자
      ✅ 완료 - pdfplumber: 1126자
   📄 페이지 297/600 처리 중...
      🔍 EasyOCR: 357자 (7.1초)
      📄 pdfplumber: 393자
      ✅ 완료 - pdfplumber: 393자
   📄 페이지 298/600 처리 중...
      🔍 EasyOCR: 713자 (9.4초)
      📄 pdfplumber: 805자
      ✅ 완료 - pdfplumber: 805자
   📄 페이지 299/600 처리 중...
      🔍 EasyOCR: 873자 (9.7초)
      📄 pdfplumber: 937자
      ✅ 완료 - pdfplumber: 937자
   📄 페이지 300/600 처리 중...
      🔍 EasyO

      🔍 EasyOCR: 1030자 (11.0초)
      📄 pdfplumber: 1107자
      ✅ 완료 - pdfplumber: 1107자
   📄 페이지 364/600 처리 중...
      🔍 EasyOCR: 1072자 (11.1초)
      📄 pdfplumber: 1134자
      ✅ 완료 - pdfplumber: 1134자
   📄 페이지 365/600 처리 중...
      🔍 EasyOCR: 604자 (8.5초)
      📄 pdfplumber: 639자
      ✅ 완료 - pdfplumber: 639자
   📄 페이지 366/600 처리 중...
      🔍 EasyOCR: 677자 (9.2초)
      📄 pdfplumber: 742자
      ✅ 완료 - pdfplumber: 742자
   📄 페이지 367/600 처리 중...
      🔍 EasyOCR: 1085자 (10.7초)
      📄 pdfplumber: 1161자
      ✅ 완료 - pdfplumber: 1161자
   📄 페이지 368/600 처리 중...
      🔍 EasyOCR: 1097자 (11.0초)
      📄 pdfplumber: 1163자
      ✅ 완료 - pdfplumber: 1163자
   📄 페이지 369/600 처리 중...
      🔍 EasyOCR: 1217자 (11.7초)
      📄 pdfplumber: 1282자
      ✅ 완료 - pdfplumber: 1282자
   📄 페이지 370/600 처리 중...
      🔍 EasyOCR: 734자 (9.4초)
      📄 pdfplumber: 773자
      ✅ 완료 - pdfplumber: 773자
   📄 페이지 371/600 처리 중...
      🔍 EasyOCR: 552자 (8.5초)
      📄 pdfplumber: 606자
      ✅ 완료 - pdfplumber: 606자
   📄 페이지 372/600 처리 중...

      🔍 EasyOCR: 607자 (8.4초)
      📄 pdfplumber: 663자
      ✅ 완료 - pdfplumber: 663자
   📄 페이지 437/600 처리 중...
      🔍 EasyOCR: 1024자 (10.1초)
      📄 pdfplumber: 1115자
      ✅ 완료 - pdfplumber: 1115자
   📄 페이지 438/600 처리 중...
      🔍 EasyOCR: 1125자 (10.8초)
      📄 pdfplumber: 1179자
      ✅ 완료 - pdfplumber: 1179자
   📄 페이지 439/600 처리 중...
      🔍 EasyOCR: 702자 (9.1초)
      📄 pdfplumber: 768자
      ✅ 완료 - pdfplumber: 768자
   📄 페이지 440/600 처리 중...
      🔍 EasyOCR: 972자 (10.0초)
      📄 pdfplumber: 1035자
      ✅ 완료 - pdfplumber: 1035자
   📄 페이지 441/600 처리 중...
      🔍 EasyOCR: 860자 (9.8초)
      📄 pdfplumber: 912자
      ✅ 완료 - pdfplumber: 912자
   📄 페이지 442/600 처리 중...
      🔍 EasyOCR: 558자 (8.3초)
      📄 pdfplumber: 635자
      ✅ 완료 - pdfplumber: 635자
   📄 페이지 443/600 처리 중...
      🔍 EasyOCR: 976자 (10.0초)
      📄 pdfplumber: 1050자
      ✅ 완료 - pdfplumber: 1050자
   📄 페이지 444/600 처리 중...
      🔍 EasyOCR: 773자 (9.3초)
      📄 pdfplumber: 804자
      ✅ 완료 - pdfplumber: 804자
   📄 페이지 445/600 처리 중...
     

      🔍 EasyOCR: 607자 (8.2초)
      📄 pdfplumber: 663자
      ✅ 완료 - pdfplumber: 663자
   📄 페이지 510/600 처리 중...
      🔍 EasyOCR: 555자 (8.7초)
      📄 pdfplumber: 591자
      ✅ 완료 - pdfplumber: 591자
   📄 페이지 511/600 처리 중...
      🔍 EasyOCR: 1036자 (10.2초)
      📄 pdfplumber: 1087자
      ✅ 완료 - pdfplumber: 1087자
   📄 페이지 512/600 처리 중...
      🔍 EasyOCR: 701자 (9.1초)
      📄 pdfplumber: 750자
      ✅ 완료 - pdfplumber: 750자
   📄 페이지 513/600 처리 중...
      🔍 EasyOCR: 615자 (9.2초)
      📄 pdfplumber: 693자
      ✅ 완료 - pdfplumber: 693자
   📄 페이지 514/600 처리 중...
      🔍 EasyOCR: 1051자 (10.5초)
      📄 pdfplumber: 1110자
      ✅ 완료 - pdfplumber: 1110자
   📄 페이지 515/600 처리 중...
      🔍 EasyOCR: 1050자 (10.5초)
      📄 pdfplumber: 1123자
      ✅ 완료 - pdfplumber: 1123자
   📄 페이지 516/600 처리 중...
      🔍 EasyOCR: 555자 (8.9초)
      📄 pdfplumber: 609자
      ✅ 완료 - pdfplumber: 609자
   📄 페이지 517/600 처리 중...
      🔍 EasyOCR: 1062자 (10.5초)
      📄 pdfplumber: 1122자
      ✅ 완료 - pdfplumber: 1122자
   📄 페이지 518/600 처리 중...
   

      🔍 EasyOCR: 929자 (9.7초)
      📄 pdfplumber: 980자
      ✅ 완료 - pdfplumber: 980자
   📄 페이지 583/600 처리 중...
      🔍 EasyOCR: 555자 (8.1초)
      📄 pdfplumber: 609자
      ✅ 완료 - pdfplumber: 609자
   📄 페이지 584/600 처리 중...
      🔍 EasyOCR: 581자 (8.9초)
      📄 pdfplumber: 571자
      ✅ 완료 - EasyOCR: 581자
   📄 페이지 585/600 처리 중...
      🔍 EasyOCR: 1009자 (10.2초)
      📄 pdfplumber: 1067자
      ✅ 완료 - pdfplumber: 1067자
   📄 페이지 586/600 처리 중...
      🔍 EasyOCR: 1075자 (10.5초)
      📄 pdfplumber: 1124자
      ✅ 완료 - pdfplumber: 1124자
   📄 페이지 587/600 처리 중...
      🔍 EasyOCR: 735자 (8.8초)
      📄 pdfplumber: 792자
      ✅ 완료 - pdfplumber: 792자
   📄 페이지 588/600 처리 중...
      🔍 EasyOCR: 412자 (7.9초)
      📄 pdfplumber: 426자
      ✅ 완료 - pdfplumber: 426자
   📄 페이지 589/600 처리 중...
      🔍 EasyOCR: 581자 (8.7초)
      📄 pdfplumber: 592자
      ✅ 완료 - pdfplumber: 592자
   📄 페이지 590/600 처리 중...
      🔍 EasyOCR: 806자 (9.6초)
      📄 pdfplumber: 824자
      ✅ 완료 - EasyOCR: 806자
   📄 페이지 591/600 처리 중...
      🔍 EasyOCR: 